import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when

create local SparkSession

In [2]:
findspark.init()

In [3]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('PySpark_Basics') \
    .getOrCreate()

read csv with inferschema

In [4]:
%%time

df = spark.read \
    .option('header', True) \
    .option('inferSchema', True) \
    .csv('../data/ds_salaries.csv')

CPU times: total: 0 ns
Wall time: 4.1 s


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [5]:
%%time

df = spark.read \
    .option('header', True) \
    .option('inferSchema', True) \
    .csv('../data/ds_salaries.csv')

CPU times: total: 0 ns
Wall time: 238 ms


write schema of scv on screen

In [6]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [4]:
schema = StructType([
    StructField('id', IntegerType(), nullable=True),
    StructField('work_year', IntegerType(), nullable=True),
    StructField('experience_level', StringType(), nullable=True),
    StructField('employment_type', StringType(), nullable=True),
    StructField('job_title', StringType(), nullable=True),
    StructField('salary', IntegerType(), nullable=True),
    StructField('salary_currency', StringType(), nullable=True),
    StructField('salary_in_usd', IntegerType(), nullable=True),
    StructField('employee_residence', StringType(), nullable=True),
    StructField('remote_ratio', IntegerType(), nullable=True),
    StructField('company_location', StringType(), nullable=True),
    StructField('company_size', StringType(), nullable=True)
])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [5]:
%%time

df = spark.read \
    .option('header', True) \
    .schema(schema) \
    .csv('../data/ds_salaries.csv')

CPU times: total: 31.2 ms
Wall time: 1.52 s


this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [6]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [7]:
df.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [8]:
display(df.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [9]:
df_job_title = df.select('job_title').distinct()

print all rows from df_job_titles without truncating jobs

In [10]:
df_job_title.show(truncate=False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [11]:
df_analyric = df \
    .groupby('job_title') \
    .agg(min('salary_in_usd').alias('min_salary'),
        max('salary_in_usd').alias('max_salary'),
        avg('salary_in_usd').alias('avg_salary'))

print all rows from df_analytic without trancating jobs

In [12]:
df_analyric.show(truncate=False)

+----------------------------------------+----------+----------+------------------+
|job_title                               |min_salary|max_salary|avg_salary        |
+----------------------------------------+----------+----------+------------------+
|3D Computer Vision Researcher           |5409      |5409      |5409.0            |
|Lead Data Engineer                      |56000     |276000    |139724.5          |
|Head of Machine Learning                |79039     |79039     |79039.0           |
|Data Specialist                         |165000    |165000    |165000.0          |
|Data Analytics Lead                     |405000    |405000    |405000.0          |
|Machine Learning Scientist              |12000     |260000    |158412.5          |
|Lead Data Analyst                       |19609     |170000    |92203.0           |
|Data Engineering Manager                |59303     |174000    |123227.2          |
|Staff Data Scientist                    |105000    |105000    |105000.0    

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

print all data from df_analytic

it isn't beautifull, so we need to put now row_id on first place in df_analytic

print df_analytic now

here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

print here df_exp_lvl

create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

print df_best

drop duplicates if exist by experience_level

print df_best

create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

print df_new_best

write df_new_best like 1.csv and load then it to df_final

print df_final

filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

print df_final

last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system